In [ ]:
import pathlib
import sys

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

sys.path.append(str(pathlib.PurePath('..', '..', 'src')))
from linear_geodesic_optimization.data import input_network, tomography
from linear_geodesic_optimization import plot

In [ ]:
path_probes = pathlib.PurePath('..', 'Internet2', 'probes.csv')
path_links = pathlib.PurePath('..', 'Internet2', 'measurements', '20240804190000.csv')
graph = input_network.get_graph_from_csvs(path_probes, path_links, clustering_distance=500000, should_compute_curvatures=False, directed=True, symmetrize=True)

routes = tomography.get_shortest_routes(graph, 'rtt')
traffic_matrix = tomography.compute_traffic_matrix(graph, routes, 'throughput')

In [ ]:
graph_curvatures = input_network.compute_ricci_curvatures(
    graph.copy(), weight_label='throughputs',
    routes=routes, traffic_matrix=traffic_matrix
)

fig, ax = plt.subplots(1, 1, facecolor='#808080')
fig.set_dpi(200)
plot.get_network_plot(graph_curvatures, ax=ax)
plt.show()

In [ ]:
graph_curvatures_optimal = input_network.compute_ricci_curvatures(
    graph.copy(), weight_label='throughputs',
    routes=routes, traffic_matrix=traffic_matrix, force_optimal_transport=True
)

fig, ax = plt.subplots(1, 1, facecolor='#808080')
fig.set_dpi(200)
plot.get_network_plot(graph_curvatures_optimal, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_dpi(150)
x = np.array([graph_curvatures.edges[edge]['ricciCurvature'] for edge in graph.edges])
y = np.array([graph_curvatures_optimal.edges[edge]['ricciCurvature'] for edge in graph.edges])
ax.plot([-7, 2], [-7, 2], 'r--')
ax.plot(
    x, y, 'b.'
)
ax.set_aspect('equal')
ax.set_xlabel('Data-Plane Transport Plan')
ax.set_ylabel('Optimal Transport Plan')
plt.show()

In [ ]:
differences = list(sorted([
    (np.abs(kappa - kappa_optimal), u, v, kappa, kappa_optimal)
    for u, v in graph.edges
    for kappa in (graph_curvatures.edges[u, v]['ricciCurvature'],)
    for kappa_optimal in (graph_curvatures_optimal.edges[u, v]['ricciCurvature'],)
], reverse=True))
print(f'{len(differences)} total edges')
print('Top 10 differences in curvature:')
for difference, u, v, kappa, kappa_optimal in differences[:10]:
    print(f'\t{u}, {v}: {difference:0.4f} ({kappa:0.4f} vs. {kappa_optimal:0.4f})')